In [ ]:
import pandas as pd
import numpy as np
import os
import sys

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
case = ['1119','910','873','1489','989','880','1282','1584','875','1009','MWB_110',
        'MWB_161','MWB_174','MWB_242','MWB_276','MWB_437','MWB_490','MWB_506',
        'MWB_540','MWB_548','MWB_564','MWB_626','MWB_719','MWB_740','MWB_746',
        'MWB_815','MWB_870','MWB_1226','MWB_1276',
        'MWB_1445']


control = ['1093','1134','1293','1316','1389','1428','1477','1481','1557','1591','MWB_895','MWB_911',
           'MWB_941','MWB_956','MWB_961',"MWB_1008",'MWB_1040','MWB_1151','MWB_1244',
           'MWB_1250','MWB_1443','MWB_1676','MWB_1960','MWB_1964','MWB_1975',
           'MWB_1977','MWB_1982','MWB_1995','MWB_2007','MWB_2008']

print(len(case))
print(len(control))

In [ ]:
# These Are the New COlumns that are used to Replace the Column Name
new_col = ['Chr', 'Start', 'End', 'Ref', 'Alt', 'Ref_Gene', 'Func_refGene',
       'ExonicFunc_refGene', 'Gene_ensGene', 'avsnp147', 'AAChange_ensGene',
       'AAChange_refGene', 'Clinvar','InterVar_Evidence','Freq_gnomAD_genome_ALL',
       'Freq_esp6500siv2_all', 'Freq_1000g2015aug_all', 'CADD_raw',
       'CADD_phred', 'SIFT_score', 'GERP++_RS', 'phyloP46way_placental',
       'dbscSNV_ADA_SCORE', 'dbscSNV_RF_SCORE', 'Interpro_domain',#
       'AAChange.knownGene', 'rmsk', 'MetaSVM_score',
       'Freq_gnomAD_genome_POPs', 'OMIM', 'Phenotype_MIM', 'OrphaNumber',
       'Orpha', 'Otherinfo']

In [ ]:
open_col_only = ['#Chr', 'Start', 'End', 'Ref', 'Alt', 'Ref.Gene', 'Func.refGene',
        'avsnp147','clinvar: Clinvar ',' InterVar: InterVar and Evidence ', 'OMIM']
# these columns are need to open while opening intervar file
# After Opening we will change the column Name
out_col = ['Chr', 'Start', 'End', 'Ref', 'Alt', 'Ref_Gene', 'Func_refGene',
        'avsnp147','Clinvar','InterVar', 'OMIM']
## 

# In Case fo SNV the start and end position is same so we dont need Start and End
# But in case of indels the insertion and deletion might not be same so we need
len(open_col_only) == len(out_col)

In [ ]:
def check_file(case_or_control, file_extension = ".hg19_multianno.txt.intervar"):
    # "Annovar_result/1119/1119.hg19_multianno.txt.intervar"
    """This Function us useful to check weather file exists or Not and return 0 if 
    any file not exists"""
    import os
    file_extension = file_extension
    for _ in case_or_control:
        
        condition = os.path.exists(_+"/"+_+file_extension)
        if condition:
            print(_, "exist", end= "-->")
        else:
            print()
            print("\n")
            print("X-#-!"*20)
            print(_,"not exist")
            print("###"*20,"\n")
            print("Check this file and run",_)
            return 0
    print("\n\nAll File exists")
    return 1

In [ ]:
check_file(case)

In [ ]:
check_file(control)

##### Since all file exists Now we need to create different file snv and Indel

In [ ]:
def split(x):
    x = x[10:]
    splitted = x.rsplit()
    if splitted[0] in ["Pathogenic", "Benign"]:
        return splitted[:1][0]
    else:
        return "_".join(splitted[:2])

# For Python Change to Code from setting
for _ in case:
    condition = os.path.exists(_+"/"+_+"_indels.csv")
    if condition:
        print("Doing", _)
        df = pd.read_csv(_+"/"+_+".hg19_multianno.txt.intervar",
                 low_memory=False,sep = "\t",usecols = open_col_only)
        df.columns = out_col
        df.Clinvar = df.Clinvar.apply(lambda x : x[9:]) # Skip name 
        df.InterVar = df.InterVar.apply(lambda x : split(x)) # Skip name Intervar

        df_snvs = df.loc[(df.Ref != "-") & (df.Alt.str.len()== 1) & 
                         (df.Ref.str.len() == 1)&(df.Alt != "-")]
        df_indels = df.loc[(df.Ref == "-") | (df.Alt == "-")]
        df_snvs.to_csv(_+"/"+_+"_snv.csv", index = False)
        df_indels.to_csv(_+"/"+_+"_indels.csv", index = False)
        
        del df
    else:
        print("skipping", _)
    print("done",_)

In [ ]:
#_ = "1119"
#df = pd.read_csv(_+"/"+_+".hg19_multianno.txt.intervar",
#                 low_memory=False,nrows = 1000,sep = "\t",usecols = open_col_only)
#df.columns = out_col
#df.Clinvar = df.Clinvar.apply(lambda x : x[9:])
#df.InterVar = df.InterVar.apply(lambda x : x[10:])
#df_snvs = df.loc[(df.Ref != "-") & (df.Alt.str.len()== 1) & (df.Ref.str.len() == 1)]
#df_indels = df.loc[(df.Ref == "-") | (df.Alt == "-")]
#df_snvs.to_csv(_+"/"+_+"_snv.csv", index = False)
#df_indels.to_csv(_+"/"+_+"_indels.csv", index = False)

In [ ]:
#df_snvs = df.loc[(df.Ref.str.len() == 1) & (df.Alt.str.len()== 1)]
#df_indels = df.loc[df.Ref.str.len()>1 | (df.Alt.str.len()>1)]

df_snvs = df.loc[(df.Ref != "-") & (df.Alt.str.len()== 1) & (df.Ref.str.len() == 1)]
df_indels = df.loc[(df.Ref == "-") | (df.Alt == "-")]

In [ ]:
#df_snvs.head()
#df_snvs.to_csv(_+"/"+_+"_snv.csv", index = False)

#  Merging Indel data as a single File

In [ ]:
case_merged = None  # To clear Memory
control_merged = None
open_cols = ["Chr", 'Start', 'Ref',"Alt"]
merged_cols = ["Chr", 'Start', 'Ref',"Alt","SNP"]
control_merged = pd.read_csv("1093/1093_indels.csv", low_memory= False,usecols=open_cols, dtype="object")
control_merged.drop_duplicates(subset = "Start", keep = False, inplace = True)
control_merged["SNP"] = 1
print(control_merged.shape)
control_merged.head()

In [ ]:
control_merged.head()